In [1]:
# Lets import the libraries
import socket, cv2, pickle, struct, imutils ,threading

In [2]:
def transmit():
    while True:
        if wclient_socket:
            url = 'http://192.168.221.154:8080/video' #url given on ip webcam application
            vid = cv2.VideoCapture(url)
            while(vid.isOpened()):
                img,frame = vid.read()
                frame = imutils.resize(frame,width=200)
                a = pickle.dumps(frame)
                message = struct.pack("Q",len(a))+a
                wclient_socket.sendall(message)
            
                cv2.imshow('TRANSMITTING VIDEO',frame)
                key = cv2.waitKey(1) & 0xFF
                if key ==ord('q'):
                    wclient_socket.close()
                    break
    cv2.destroyAllWindows()
    
def recieve(data,payload_size):
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) 
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]
        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data)
        frame = imutils.resize(frame,height=1020,width=640)
        cv2.imshow("RECEIVING VIDEO",frame)
        key = cv2.waitKey(1) & 0xFF
        if key  == ord('q'):
            break
    client_socket.close()
    cv2.destroyAllWindows()

In [3]:
# connect to windows socket
client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_ip = '192.168.99.1' #IP Address of Host to be Entered 
port = 9999
client_socket.connect((host_ip,port))

In [4]:
# Create Socket
client_socket2 = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_name  = socket.gethostname()
host_ip = socket.gethostbyname(host_name)
port2 = 6666
socket_address = (host_ip,port2)
client_socket2.bind(socket_address)

In [5]:
# Socket Listen and accept waiting to windows to connect
client_socket2.listen(5)
print("LISTENING AT:",socket_address)
wclient_socket,addr = client_socket2.accept()
print('GOT CONNECTION FROM:',addr)

LISTENING AT: ('192.168.99.1', 6666)
GOT CONNECTION FROM: ('192.168.99.1', 64744)


In [6]:
data = b""
payload_size = struct.calcsize("Q")
x1=threading.Thread(target=recieve , args=(data , payload_size))
x2=threading.Thread(target=transmit)
x1.start()
x2.start()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4ce32bd7ad77>", line 28, in recieve
struct.error: unpack requires a buffer of 8 bytes
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4ce32bd7ad77>", line 11, in transmit
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
